1.Scrapping data from wbesite

1. Importing libraries essetial to scrapping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

2. Pointing url to fetch data from

In [2]:
url = 'https://www.pig333.com/markets_and_prices/'
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

3. Locating tags where the data in relevant fetching into a dataframe

In [3]:
markets = []
listings = soup.find('div', {'class': 'contingut'}).find_all('div', {'class': 'preu_mercat'})

In [4]:
print(markets)

[]


4. Fetching data from located tags in the HTML into a proper dataframe.

In [5]:
for item in listings:
    market = {}

    market['country'] = item.find('a', {'class': 'titol'}).getText()
    market['date'] = item.select('.col-titol > span.data')[0].getText()
    market['price'] = item.find('span', {'class': 'preu'}).getText()
    market['CUR'] = item.find('span', {'class': 'moneda'}).getText()
    market['unit'] = item.find('span', {'class': 'unitats'}).getText()
    market['reference'] = item.find('span', {'class': 'data fs9'}).getText()
    market['variation'] = item.find('span', {'class': 'perc'}).getText()

    markets.append(market)
  

pricing = pd.DataFrame(markets)
pricing

,country,date,price,CUR,unit,reference,variation
0,Belgium - Danis,6 Aug,0.960,EUR,kg,Live,0%
1,Denmark,6 Aug,10.50,DKK,kg,Carcass 60%,0%
2,France - MPB,3 Aug,1.293,EUR,kg,Carcass 56 TMV,0%
3,Germany,5 Aug,1.470,EUR,kg,Carcass 57%,0%
4,Italy,6 Aug,1.325,EUR,kg,160/176 kg Live,3.9%
5,Netherlands - Beursprijs,31 Jul,1.050,EUR,kg,Live,0%
6,Poland,31 Jul,6.441,PLN,kg,Carcass 57%,2.9%
7,Romania,24 Jul,536.79,RON,100kg,Live,2.0%
8,Russia,29 Jul,112.20,RUB,kg,Live,6.8%
9,Spain - Lleida,6 Aug,1.300,EUR,kg,Live,0%


In [6]:
print(market)

{'country': 'South Korea', 'date': '6 Aug', 'price': '4279', 'CUR': 'KRW', 'unit': 'kg', 'reference': 'Carcass', 'variation': ' 2.7%'}


In [7]:
pricing['price'] = pricing['price'].astype(float)

5. Create Unit equalizing column

Find the price units that  distorts (dressing percentage) the value of the dafult price of a pig at the gate and equalize it for a price per kilo to into all currencies

In [8]:
treat_price=pricing.loc[(pricing['unit']=='100kg')]
treat_price

,country,date,price,CUR,unit,reference,variation
7,Romania,24 Jul,536.79,RON,100kg,Live,2.0%
15,Canada - Ontario,31 Jul,127.01,CAD,100kg,Carcass,6.1%
16,Canada - Quebec,7 Aug,148.18,CAD,100kg,Carcass,4.9%


In [9]:
treat_price.price

7     536.79
15    127.01
16    148.18
Name: price, dtype: float64

In [10]:
correct_price = treat_price.price.div(100)
correct_price

7     5.3679
15    1.2701
16    1.4818
Name: price, dtype: float64

In [11]:
treat_price2=pricing.loc[(pricing['unit']=='cwt')]
treat_price2

,country,date,price,CUR,unit,reference,variation
20,USA - Iowa/Minnesota,6 Aug,38.81,USD,cwt,Carcass,2.0%


In [12]:
correct_price2 = treat_price2.price.div(50.802)
correct_price2

20    0.763946
Name: price, dtype: float64

In [13]:
treat_price3=pricing.loc[(pricing['CUR']=='GBX')]
treat_price3

,country,date,price,CUR,unit,reference,variation
10,United Kingdom,1 Aug,164.68,GBX,kg,SPP Euro Spec,0.3%


In [14]:
correct_price3 = treat_price3.price.div(100)
correct_price3

10    1.6468
Name: price, dtype: float64

In [15]:
pricing.update(correct_price)

In [16]:
pricing.update(correct_price2)

In [17]:
pricing.update(correct_price3)

In [18]:
pricing

,country,date,price,CUR,unit,reference,variation
0,Belgium - Danis,6 Aug,0.960000,EUR,kg,Live,0%
1,Denmark,6 Aug,10.500000,DKK,kg,Carcass 60%,0%
2,France - MPB,3 Aug,1.293000,EUR,kg,Carcass 56 TMV,0%
3,Germany,5 Aug,1.470000,EUR,kg,Carcass 57%,0%
4,Italy,6 Aug,1.325000,EUR,kg,160/176 kg Live,3.9%
5,Netherlands - Beursprijs,31 Jul,1.050000,EUR,kg,Live,0%
6,Poland,31 Jul,6.441000,PLN,kg,Carcass 57%,2.9%
7,Romania,24 Jul,5.367900,RON,100kg,Live,2.0%
8,Russia,29 Jul,112.200000,RUB,kg,Live,6.8%
9,Spain - Lleida,6 Aug,1.300000,EUR,kg,Live,0%


In [19]:
##4.1 exporting prices for swine into a CSV file

In [21]:
#import datetime

#current_date = datetime.datetime.now()
#filename = str(current_date.day)+str(current_date.month)+str(current_date.year)
#pricing.to_csv(str('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetoday'+filename+ '.csv'))
#pricing.to_csv('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetoday.csv')

6. Creat Currency equalizing column

6.1 Finding the currency conversions on a website online to keep the values always updated with FX figures

In [22]:
api_url = "https://openexchangerates.org/api/latest.json?app_id=84f0e7f007024e6aa0c5fc52f419b104"
r = requests.get(api_url)

r.raise_for_status()
result = r.json()
rates = result['rates']
print(rates)

{'AED': 3.6732, 'AFN': 76.960585, 'ALL': 104.84989, 'AMD': 481.616228, 'ANG': 1.799535, 'AOA': 582.5, 'ARS': 72.890768, 'AUD': 1.396229, 'AWG': 1.8002, 'AZN': 1.7025, 'BAM': 1.658404, 'BBD': 2, 'BDT': 85.002098, 'BGN': 1.6595, 'BHD': 0.377934, 'BIF': 1933.083753, 'BMD': 1, 'BND': 1.374369, 'BOB': 6.922089, 'BRL': 5.4376, 'BSD': 1, 'BTC': 8.3408984e-05, 'BTN': 75.078502, 'BWP': 11.766676, 'BYN': 2.457009, 'BZD': 2.020749, 'CAD': 1.33781, 'CDF': 1976.961192, 'CHF': 0.913307, 'CLF': 0.028224, 'CLP': 788.79937, 'CNH': 6.966504, 'CNY': 6.9673, 'COP': 3752.895973, 'CRC': 591.136287, 'CUC': 1, 'CUP': 25.75, 'CVE': 94.37, 'CZK': 22.307001, 'DJF': 178.432371, 'DKK': 6.31735, 'DOP': 58.537022, 'DZD': 128.513438, 'EGP': 15.9553, 'ERN': 15.003119, 'ETB': 35.674655, 'EUR': 0.848088, 'FJD': 2.1178, 'FKP': 0.765239, 'GBP': 0.765239, 'GEL': 3.085, 'GGP': 0.765239, 'GHS': 5.784361, 'GIP': 0.765239, 'GMD': 51.87, 'GNF': 9671.301265, 'GTQ': 7.719181, 'GYD': 209.456937, 'HKD': 7.75055, 'HNL': 24.756107, '

In [23]:
usdrates = pd.DataFrame([rates])
usdrates.T.reset_index()

,index,0
0,AED,3.673200
1,AFN,76.960585
2,ALL,104.849890
3,AMD,481.616228
4,ANG,1.799535
...,...,...
166,XPT,0.001022
167,YER,250.350066
168,ZAR,17.615851
169,ZMW,18.471232


6.2 Selecting only the currencys I actually need conversion for the quotations I have in each country

Filtering the ACTUAL CURRENCIES I need (16 only of actual_currencies) from the total FX rates I've have caputred

In [24]:
actual_currency = pricing['CUR']
actual_currency[10]='GBP'
actual_currency

/Users/lstorino/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0     EUR
1     DKK
2     EUR
3     EUR
4     EUR
5     EUR
6     PLN
7     RON
8     RUB
9     EUR
10    GBP
11    UAH
12    ARS
13    BRL
14    BRL
15    CAD
16    CAD
17    CLP
18    COP
19    MXN
20    USD
21    CNY
22    KRW
Name: CUR, dtype: object

In [25]:
actual_currency_list = list(actual_currency)
usd_daily_rate=usdrates[actual_currency_list]
usd_d_rates = usd_daily_rate.T.reset_index()

In [26]:
usd_d_rates

,index,0
0,EUR,0.848088
1,DKK,6.317350
2,EUR,0.848088
3,EUR,0.848088
4,EUR,0.848088
5,EUR,0.848088
6,PLN,3.743633
7,RON,4.105500
8,RUB,73.309900
9,EUR,0.848088


In [27]:
usd_d_rates.rename(columns={'index': 'CUR', 0: 'Rate'},inplace = True)
usd_d_rates

,CUR,Rate
0,EUR,0.848088
1,DKK,6.317350
2,EUR,0.848088
3,EUR,0.848088
4,EUR,0.848088
5,EUR,0.848088
6,PLN,3.743633
7,RON,4.105500
8,RUB,73.309900
9,EUR,0.848088


In [28]:
pricing

,country,date,price,CUR,unit,reference,variation
0,Belgium - Danis,6 Aug,0.960000,EUR,kg,Live,0%
1,Denmark,6 Aug,10.500000,DKK,kg,Carcass 60%,0%
2,France - MPB,3 Aug,1.293000,EUR,kg,Carcass 56 TMV,0%
3,Germany,5 Aug,1.470000,EUR,kg,Carcass 57%,0%
4,Italy,6 Aug,1.325000,EUR,kg,160/176 kg Live,3.9%
5,Netherlands - Beursprijs,31 Jul,1.050000,EUR,kg,Live,0%
6,Poland,31 Jul,6.441000,PLN,kg,Carcass 57%,2.9%
7,Romania,24 Jul,5.367900,RON,100kg,Live,2.0%
8,Russia,29 Jul,112.200000,RUB,kg,Live,6.8%
9,Spain - Lleida,6 Aug,1.300000,EUR,kg,Live,0%


In [29]:
import seaborn as sns

In [30]:
pricing['usd_base_kg'] = pricing.price.div(usd_d_rates.Rate)
pig_price_kg_usd = pricing[['country','date','usd_base_kg']]
pig_price_kg_usd.style.hide_index().highlight_max(color = 'yellow').highlight_min(color = 'lightblue')

country,date,usd_base_kg
Belgium - Danis,6 Aug,1.131958
Denmark,6 Aug,1.662089
France - MPB,3 Aug,1.524606
Germany,5 Aug,1.733311
Italy,6 Aug,1.562338
Netherlands - Beursprijs,31 Jul,1.238079
Poland,31 Jul,1.720521
Romania,24 Jul,1.307490
Russia,29 Jul,1.530489
Spain - Lleida,6 Aug,1.532860


In [31]:
pig_price_kg_usd.style.background_gradient(cmap = 'Greens')

,country,date,usd_base_kg
0,Belgium - Danis,6 Aug,1.131958
1,Denmark,6 Aug,1.662089
2,France - MPB,3 Aug,1.524606
3,Germany,5 Aug,1.733311
4,Italy,6 Aug,1.562338
5,Netherlands - Beursprijs,31 Jul,1.238079
6,Poland,31 Jul,1.720521
7,Romania,24 Jul,1.307490
8,Russia,29 Jul,1.530489
9,Spain - Lleida,6 Aug,1.532860


In [32]:
usd_pig_price_blue = pig_price_kg_usd.sort_values(by = 'usd_base_kg').style.bar(color= 'lightblue')

In [33]:
usd_pig_price_blue

,country,date,usd_base_kg
20,USA - Iowa/Minnesota,6 Aug,0.763946
12,Argentine,31 Jul,0.902858
15,Canada - Ontario,31 Jul,0.949387
16,Canada - Quebec,7 Aug,1.107631
13,Brazil - Paraná,3 Aug,1.129175
0,Belgium - Danis,6 Aug,1.131958
19,Mexico - Sonora,31 Jul,1.134944
14,Brazil - Santa Catarina,6 Aug,1.215610
5,Netherlands - Beursprijs,31 Jul,1.238079
7,Romania,24 Jul,1.307490


In [34]:
import datetime

current_date = datetime.datetime.now()
filename = str(current_date.day)+str(current_date.month)+str(current_date.year)
usd_pig_price_blue.to_excel(str('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetodayUSDBlueGrade'+filename+ '.xls'))
#pricing.to_csv('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetoday.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetodayUSDBlueGrade1082020.xls'

In [ ]:
pig_price_kg_usd.describe()

8. Export to CSV file for further temporal analisys

In [ ]:
import datetime

current_date = datetime.datetime.now()
filename = str(current_date.day)+str(current_date.month)+str(current_date.year)
pig_price_kg_usd.to_csv(str('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetodayUSD'+filename+ '.csv'))
#pricing.to_csv('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetoday.csv')